In [ ]:
from pychomp import *
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

In [ ]:
def hopf ( X, a):
    sigma = -1.
    a, = params
    #a,sigma = params
    return [a*X[0]-X[1]+sigma*X[0]*(X[0]**2+X[1]**2), 
            X[0]+a*X[1]+sigma*X[1]*(X[0]**2+X[1]**2)]

In [ ]:
ODE_model = lambda x, params: hopf( x, params )
xmax,ymax=1.5,1.5
xmin,ymin = -1.5,-1.5
bounds = [[xmin,xmax],[ymin,ymax]]
boxes = [10,10]
num_samples = 3
params = [1.0]

In [ ]:
phase_complex = GeometricCubicalComplex(bounds,boxes)
phase_complex.delta

In [ ]:
X = phase_complex.complex

In [ ]:
x_vert = [xmin+i*phase_complex.delta[0] for i in range(boxes[0]+1)]
y_vert = [ymin+i*phase_complex.delta[1] for i in range(boxes[1]+1)]
#vertices

In [ ]:
C, discrete_flow = TransversalityComplex(phase_complex, ODE_model,params,num_samples)
FC = FlowComplex(C, discrete_flow)

In [ ]:
# initial condition
x = [.5,.5]

# time points
t = np.linspace(0,20)

# solve ODE
y = integrate.odeint(ODE_model,x,t)

# plot results
plt.plot(t,y)
plt.xlabel('time')
plt.ylabel('y(t)')
plt.show()

In [ ]:
""""
I need to define the possible values my initial points will take as they
relate to the equilibrium point. In this case I chose to plot 10 trajectories
ranging from 0.1 to 5
"""
#values = np.linspace(-1.25, 1.25, 10)
# I want each trajectory to have a different color
#vcolors = plt.cm.autumn_r(np.linspace(0.1, 1, len(values)))
 
# Open figure
f = plt.figure()
"""
I need to define a range of time over which to integrate the system of ODEs
The values don't really matter in this case because our system doesn't have t
on the right hand side of dx/dt and dy/dt, but it is a necessary input for
integrate.odeint.
"""

# Get limits of trajectory plot
#xmax,ymax = 1.25,1.25
#ymax = plt.ylim(ymin=0)[1]
#xmax = plt.xlim(xmin=0)[1]
# Define number of points
nb_points = 20
# Define x and y ranges
x = np.linspace(xmin, xmax, nb_points)
y = np.linspace(ymin, ymax, nb_points)
# Create meshgrid
X1 , Y1 = np.meshgrid(x,y)
# Calculate growth rate at each grid point
DX1, DY1 = hopf([X1, Y1],params)
# Direction at each grid point is the hypotenuse of the prey direction and the
# predator direction.
M = (np.hypot(DX1, DY1))
# This is to avoid any divisions when normalizing
M[ M == 0] = 1.
# Normalize the length of each arrow (optional)
DX1 /= M
DY1 /= M
 
plt.title('Trajectories and direction fields')
"""
This is using the quiver function to plot the field of arrows using DX1 and
DY1 for direction and M for speed
"""
Q = plt.quiver(X1, Y1, DX1, DY1, M, pivot='mid', cmap=plt.cm.plasma)
plt.xlabel('Prey abundance')
plt.ylabel('Predator abundance')
#plt.legend(bbox_to_anchor=(1.05, 1.0))
plt.grid(True)
plt.xticks(x_vert)
plt.yticks(y_vert)
#plt.grid()
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.show()

In [ ]:
x_pos = []
y_pos = []
x_dir = []
y_dir = []
for u in discrete_flow.keys():
    if X.rightfringe(u): continue
    u_barys = [(bounds[1]+bounds[0])/2 for bounds in phase_complex.geometry(u)]
    #print(u,u_barys)
    for v in discrete_flow[u]:
        if X.rightfringe(v): continue
        v_barys = [(bounds[1]+bounds[0])/2 for bounds in phase_complex.geometry(v)]
        x_pos . append (u_barys[0])
        y_pos . append (u_barys[1])
        x_dir . append (v_barys[0]-u_barys[0])
        y_dir . append (v_barys[1]-u_barys[1])
        #print(v_barys)
    #print("----")

In [ ]:
plt.title('Directed Graph')
"""
This is using the quiver function to plot the field of arrows using DX1 and
DY1 for direction and M for speed
"""
Q = plt.quiver(x_pos, y_pos, x_dir, y_dir,scale=7)
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.xticks(x_vert)
plt.yticks(y_vert)
#plt.grid()
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.show()

In [ ]:
GC = GradedComplexObj.induce_from_flow(FC.complex, lambda x: FC.discrete_flow[x])

In [ ]:
DrawGradedComplex(GC.graded_complex,GC.poset)

In [ ]:
conley_complex = ConnectionMatrix(GC.graded_complex)
poset = GC.poset

In [ ]:
DrawGradedComplex(conley_complex,poset)

In [ ]:
def InducedPoset( G, predicate ):
    result = DirectedAcyclicGraph()
    S = set([v for v in G.vertices() if predicate(v)])
    for v in S:
        result.add_vertex(v)
    for v in S:
        for u in G.children(v):
            if u in S and u != v:
                result.add_edge(v,u)
    return Poset(result)#type(poset)

CMG = InducedPoset(poset, lambda v : v in conley_complex.count())

In [ ]:
#del conleyindices[fringenode]
CMG = InducedPoset(poset, lambda v : v in conley_complex.count())

In [ ]:
CMG.vertices()

In [ ]:
type(CMG)